In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as tud

from collections import Counter
import numpy as np
import random
import math

import pandas as pd
import scipy
import sklearn
from sklearn.metrics.pairwise import cosine_similarity

USE_CUDA=torch.cuda.is_available()
print(USE_CUDA)
random.seed(1)
np.random.seed(1)
torch.manual_seed(1)##freeze seed,使每次训练出来的结果都一样。
if USE_CUDA:
    torch.cuda.manual_seed(1)
    
##设置一些超参数
C=3  ##中心词周围的单词数 context words
K=100  ##negative samples
NUM_EPOCHS=2
MAX_VOCAB_SIZE=30000
LEARNING_RATE=0.2
EMBEDDING_SIZE=100
BATCH_SIZE=32
LOG_FILE = "word-embedding.log"


def word_tokenize(text):###分割单词
    return text.split()

with open("E:\\BaiduNetdiskDownload\\text8\\text8.train.txt","r") as fin:
    text=fin.read()

##text[:2000].split()
text=[w for w in word_tokenize(text.lower())]
vocab=dict(Counter(text).most_common(MAX_VOCAB_SIZE-1))##列举出前(MAX_VOCAB_SIZE-1)个单词和它出现的次数
vocab["<unk>"]=len(text)-np.sum(list(vocab.values()))##计算所有不常用词一共出现的总次数

idx_to_word=[word for word in vocab.keys()]
word_to_idx={word: i for i,word in enumerate(idx_to_word)}
##vocab.keys() 
##idx_to_word  ##每个idx对应的word
##word_to_idx  ##每个word对应的idx
##list(word_to_idx.items())[:100]##前100个热词的list
word_counts=np.array([count for count in vocab.values()],dtype=np.float32)##计算每个词出现的次数
word_freqs=word_counts/np.sum(word_counts)##计算每个词出现的频率
word_freqs=word_freqs**(3/4)
word_freqs = word_freqs / np.sum(word_freqs) # 用来做 negative sampling
##word_freqs

True


In [30]:
word_to_idx["<unk>"]

29999

In [10]:
idx_to_word

['the',
 'of',
 'and',
 'one',
 'in',
 'a',
 'to',
 'zero',
 'nine',
 'two',
 'is',
 'as',
 'eight',
 'for',
 's',
 'five',
 'three',
 'was',
 'by',
 'that',
 'four',
 'six',
 'seven',
 'with',
 'on',
 'are',
 'it',
 'from',
 'or',
 'his',
 'an',
 'be',
 'this',
 'which',
 'at',
 'he',
 'not',
 'also',
 'have',
 'were',
 'has',
 'but',
 'other',
 'their',
 'its',
 'they',
 'first',
 'some',
 'had',
 'more',
 'all',
 'can',
 'most',
 'been',
 'such',
 'many',
 'who',
 'new',
 'there',
 'used',
 'after',
 'when',
 'time',
 'into',
 'these',
 'only',
 'american',
 'see',
 'may',
 'than',
 'i',
 'world',
 'would',
 'b',
 'no',
 'd',
 'however',
 'between',
 'about',
 'over',
 'states',
 'years',
 'people',
 'if',
 'war',
 'during',
 'known',
 'united',
 'called',
 'use',
 'th',
 'system',
 'often',
 'so',
 'state',
 'history',
 'will',
 'up',
 'while',
 'where',
 'city',
 'being',
 'any',
 'then',
 'out',
 'under',
 'both',
 'made',
 'english',
 'e',
 'well',
 'them',
 'number',
 'governme

<font color="#dd0000"><font size="8">实现Dataloader：</font>

一个dataloader需要以下内容：

把所有text编码成数字，然后用subsampling预处理这些文字。

保存vocabulary，单词count，normalized word frequency

每个iteration sample一个中心词

根据当前的中心词返回context单词

根据中心词sample一些negative单词

返回单词的counts

这里有一个好的tutorial介绍如何使用PyTorch dataloader. 为了使用dataloader，我们需要定义以下两个function:

__len__ function需要返回整个数据集中有多少个item

__get__ 根据给定的index返回一个item

有了dataloader之后，我们可以轻松随机打乱整个数据集，拿到一个batch的数据等等。   


In [6]:
class WordEmbeddingDataset(tud.Dataset):
    def __init__(self,text,word_to_idx,idx_to_word,word_freqs,word_counts):
        super(WordEmbeddingDataset,self).__init__()
        self.text_encoded=[word_to_idx.get(word,word_to_idx["<unk>"]) for word in text]
        self.text_encoded=torch.Tensor(self.text_encoded)
        self.word_to_idx=word_to_idx
        self.idx_to_word=idx_to_word
        self.word_freqs=torch.Tensor(word_freqs)
        self.word_counts=torch.Tensor(word_counts)
        
    def __len__(self):
        ##这个数据集一共有多少item
        return len(self.text_encoded)
    
    def __getitem__(self,idx):
        center_word=self.text_encoded[idx]
        pos_indices=list(range(idx-C, idx))+list(range(idx+1,idx+C+1))##window内单词的idx
        pos_indices=[i%len(self.text_encoded) for i in pos_indices]##取余，防止超出text长度
        pos_words=self.text_encoded[pos_indices]##周围单词
        neg_words=torch.multinomial(self.word_freqs,K*pos_words.shape[0],True)##负例采样：在30000个单词中，对于每一个word_freqs做采样，共采样K*pos_words个不正确的单词
        return center_word,pos_words,neg_words
    
##创建dataset和dataloader
##该接口的目的：将自定义的Dataset根据batch size大小、是否shuffle等封装成一个Batch Size大小的Tensor，用于后面的训练。
dataset=WordEmbeddingDataset(text,word_to_idx,idx_to_word,word_freqs,word_counts)
dataloader=tud.DataLoader(dataset,batch_size=BATCH_SIZE,shuffle=True,num_workers=0)
dataset.text_encoded[:100]
##next(iter(dataloader))

tensor([4.8130e+03, 3.1390e+03, 1.1000e+01, 5.0000e+00, 1.9400e+02, 1.0000e+00,
        3.0150e+03, 4.6000e+01, 5.9000e+01, 1.5500e+02, 1.2700e+02, 7.4100e+02,
        4.6100e+02, 1.0485e+04, 1.3300e+02, 0.0000e+00, 2.5752e+04, 1.0000e+00,
        0.0000e+00, 1.0800e+02, 8.3300e+02, 2.0000e+00, 0.0000e+00, 1.6267e+04,
        2.9999e+04, 1.0000e+00, 0.0000e+00, 1.5200e+02, 8.3300e+02, 3.4930e+03,
        0.0000e+00, 1.9400e+02, 1.0000e+01, 1.8600e+02, 5.9000e+01, 4.0000e+00,
        5.0000e+00, 1.0620e+04, 2.1300e+02, 6.0000e+00, 1.3320e+03, 1.0200e+02,
        4.3700e+02, 1.9000e+01, 5.9000e+01, 2.7640e+03, 3.5500e+02, 6.0000e+00,
        3.6250e+03, 0.0000e+00, 7.0900e+02, 1.0000e+00, 3.6400e+02, 2.6000e+01,
        4.0000e+01, 3.7000e+01, 5.3000e+01, 5.2700e+02, 9.7000e+01, 1.1000e+01,
        5.0000e+00, 1.3980e+03, 2.9290e+03, 1.8000e+01, 5.6200e+02, 6.9100e+02,
        6.6440e+03, 0.0000e+00, 2.5200e+02, 4.8130e+03, 1.0000e+01, 1.0430e+03,
        2.7000e+01, 0.0000e+00, 3.1600e+

In [7]:
text_encoded=[word_to_idx.get(word,word_to_idx["<unk>"]) for word in text]##遍历text中所有的words,在word_to_idx中的返回其在word_to_idx的idx，否则返回29999，即<unk>9
text_encoded

[4813,
 3139,
 11,
 5,
 194,
 1,
 3015,
 46,
 59,
 155,
 127,
 741,
 461,
 10485,
 133,
 0,
 25752,
 1,
 0,
 108,
 833,
 2,
 0,
 16267,
 29999,
 1,
 0,
 152,
 833,
 3493,
 0,
 194,
 10,
 186,
 59,
 4,
 5,
 10620,
 213,
 6,
 1332,
 102,
 437,
 19,
 59,
 2764,
 355,
 6,
 3625,
 0,
 709,
 1,
 364,
 26,
 40,
 37,
 53,
 527,
 97,
 11,
 5,
 1398,
 2929,
 18,
 562,
 691,
 6644,
 0,
 252,
 4813,
 10,
 1043,
 27,
 0,
 316,
 247,
 29999,
 2964,
 789,
 189,
 4813,
 11,
 5,
 201,
 569,
 10,
 0,
 1107,
 19,
 2581,
 25,
 8819,
 2,
 273,
 31,
 4089,
 140,
 58,
 25,
 6494,
 4195,
 1,
 153,
 32,
 355,
 4813,
 37,
 1128,
 6,
 441,
 344,
 1794,
 19,
 4682,
 0,
 6645,
 1,
 7542,
 1756,
 567,
 0,
 94,
 0,
 252,
 11304,
 11,
 52,
 6644,
 89,
 26,
 263,
 36,
 5966,
 4733,
 21915,
 28,
 29999,
 41,
 315,
 5,
 25163,
 517,
 7542,
 364,
 4,
 259,
 1,
 153,
 25,
 1178,
 11,
 7542,
 201,
 1576,
 2,
 15736,
 339,
 1756,
 6644,
 4682,
 344,
 760,
 162,
 402,
 5686,
 751,
 1,
 4180,
 1104,
 4288,
 1607,
 2,
 562,
 7

In [8]:
class EmbeddingModel(nn.Module):
    def __init__(self,vocab_size,embed_size):
        super(EmbeddingModel,self).__init__()
        self.vocab_size=vocab_size
        self.embed_size=embed_size
        initrange = 0.5 / self.embed_size
        self.in_embed=nn.Embedding(self.vocab_size,self.embed_size)
        self.out_embed=nn.Embedding(self.vocab_size,self.embed_size)
        self.in_embed = nn.Embedding(self.vocab_size, self.embed_size, sparse=False)
        self.in_embed.weight.data.uniform_(-initrange, initrange)
        
    def forward(self, input_labels, pos_labels, neg_labels):
        '''
        input_labels: 中心词, [batch_size]
        pos_labels: 中心词周围 context window 出现过的单词 [batch_size * (window_size * 2)]
        neg_labelss: 中心词周围没有出现过的单词，从 negative sampling 得到 [batch_size, (window_size * 2 * K)]
        
        return: loss, [batch_size]
        '''
        
        batch_size = input_labels.size(0)
        input_embedding = self.in_embed(input_labels) # B * embed_size
        pos_embedding = self.out_embed(pos_labels) # B * (2*C) * embed_size
        neg_embedding = self.out_embed(neg_labels) # B * (2*C * K) * embed_size
        log_pos = torch.bmm(pos_embedding, input_embedding.unsqueeze(2)).squeeze() # B * (2*C)
        log_neg = torch.bmm(neg_embedding, -input_embedding.unsqueeze(2)).squeeze() # B * (2*C*K)
        log_pos = F.logsigmoid(log_pos).sum(1)
        log_neg = F.logsigmoid(log_neg).sum(1) # batch_size
        loss = log_pos + log_neg
        
        return -loss
    
    def input_embeddings(self):
        return self.in_embed.weight.data.cpu().numpy()
    
model = EmbeddingModel(MAX_VOCAB_SIZE, EMBEDDING_SIZE)
if USE_CUDA:
    model = model.cuda()

评估模型代码



In [9]:
def evaluate(filename, embedding_weights): 
    if filename.endswith(".csv"):
        data = pd.read_csv(filename, sep=",")
    else:
        data = pd.read_csv(filename, sep="\t")
    human_similarity = []
    model_similarity = []
    for i in data.iloc[:, 0:2].index:
        word1, word2 = data.iloc[i, 0], data.iloc[i, 1]
        if word1 not in word_to_idx or word2 not in word_to_idx:
            continue
        else:
            word1_idx, word2_idx = word_to_idx[word1], word_to_idx[word2]
            word1_embed, word2_embed = embedding_weights[[word1_idx]], embedding_weights[[word2_idx]]
            model_similarity.append(float(sklearn.metrics.pairwise.cosine_similarity(word1_embed, word2_embed)))
            human_similarity.append(float(data.iloc[i, 2]))

    return scipy.stats.spearmanr(human_similarity, model_similarity)# , model_similarity

def find_nearest(word):
    index = word_to_idx[word]
    embedding = embedding_weights[index]
    cos_dis = np.array([scipy.spatial.distance.cosine(e, embedding) for e in embedding_weights])
    return [idx_to_word[i] for i in cos_dis.argsort()[:10]]



In [10]:
optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATE)
for e in range(NUM_EPOCHS):
    for i, (input_labels, pos_labels, neg_labels) in enumerate(dataloader):     
        # TODO
        input_labels = input_labels.long()
        pos_labels = pos_labels.long()
        neg_labels = neg_labels.long()
        if USE_CUDA:
            input_labels = input_labels.cuda()
            pos_labels = pos_labels.cuda()
            neg_labels = neg_labels.cuda()
            
        optimizer.zero_grad()
        loss = model(input_labels, pos_labels, neg_labels).mean()
        loss.backward()
        optimizer.step()

        if i % 100 == 0:
            with open(LOG_FILE, "a") as fout:
                fout.write("epoch: {}, iter: {}, loss: {}\n".format(e, i, loss.item()))
                print("epoch: {}, iter: {}, loss: {}".format(e, i, loss.item()))
            
        
        if i % 2000 == 0:
            embedding_weights = model.input_embeddings()
            sim_simlex = evaluate("E:\\simlex-999.txt", embedding_weights)
            sim_men = evaluate("E:\\men.txt", embedding_weights)
            sim_353 = evaluate("E:\\wordsim353.csv", embedding_weights)
            with open(LOG_FILE, "a") as fout:
                print("epoch: {}, iteration: {}, simlex-999: {}, men: {}, sim353: {}, nearest to monster: {}\n".format(
                    e, i, sim_simlex, sim_men, sim_353, find_nearest("monster")))
                fout.write("epoch: {}, iteration: {}, simlex-999: {}, men: {}, sim353: {}, nearest to monster: {}\n".format(e, i, sim_simlex, sim_men, sim_353, find_nearest("monster")))
                
    embedding_weights = model.input_embeddings()
    np.save("embedding-{}".format(EMBEDDING_SIZE), embedding_weights)
    torch.save(model.state_dict(), "embedding-{}.th".format(EMBEDDING_SIZE))

epoch: 0, iter: 0, loss: 420.0445556640625
epoch: 0, iteration: 0, simlex-999: SpearmanrResult(correlation=-0.03000758531093896, pvalue=0.35377456613068947), men: SpearmanrResult(correlation=0.010178476425608098, pvalue=0.604551171264864), sim353: SpearmanrResult(correlation=-0.027965544388339973, pvalue=0.6193086287850138), nearest to monster: ['monster', 'segregated', 'transverse', 'ipx', 'famers', 'cheyenne', 'silurian', 'levy', 'exterminate', 'conjunctions']

epoch: 0, iter: 100, loss: 385.0720520019531
epoch: 0, iter: 200, loss: 328.2554931640625
epoch: 0, iter: 300, loss: 265.14727783203125
epoch: 0, iter: 400, loss: 240.17425537109375
epoch: 0, iter: 500, loss: 213.85015869140625
epoch: 0, iter: 600, loss: 187.87640380859375
epoch: 0, iter: 700, loss: 167.33200073242188
epoch: 0, iter: 800, loss: 135.29379272460938
epoch: 0, iter: 900, loss: 138.7002716064453
epoch: 0, iter: 1000, loss: 154.80137634277344
epoch: 0, iter: 1100, loss: 89.15957641601562
epoch: 0, iter: 1200, loss: 

epoch: 0, iter: 12100, loss: 32.5671501159668
epoch: 0, iter: 12200, loss: 44.96467971801758
epoch: 0, iter: 12300, loss: 32.252342224121094
epoch: 0, iter: 12400, loss: 31.49551010131836
epoch: 0, iter: 12500, loss: 44.467018127441406
epoch: 0, iter: 12600, loss: 57.43778991699219
epoch: 0, iter: 12700, loss: 32.33611297607422
epoch: 0, iter: 12800, loss: 44.452980041503906
epoch: 0, iter: 12900, loss: 33.39793395996094
epoch: 0, iter: 13000, loss: 33.874847412109375
epoch: 0, iter: 13100, loss: 32.21567153930664
epoch: 0, iter: 13200, loss: 43.80937194824219
epoch: 0, iter: 13300, loss: 55.82280349731445
epoch: 0, iter: 13400, loss: 44.71875762939453
epoch: 0, iter: 13500, loss: 32.25624084472656
epoch: 0, iter: 13600, loss: 31.146358489990234
epoch: 0, iter: 13700, loss: 32.502220153808594
epoch: 0, iter: 13800, loss: 32.64338684082031
epoch: 0, iter: 13900, loss: 44.74805450439453
epoch: 0, iter: 14000, loss: 31.94913673400879
epoch: 0, iteration: 14000, simlex-999: SpearmanrResult

epoch: 0, iter: 24100, loss: 44.669349670410156
epoch: 0, iter: 24200, loss: 32.666358947753906
epoch: 0, iter: 24300, loss: 31.057336807250977
epoch: 0, iter: 24400, loss: 32.79418182373047
epoch: 0, iter: 24500, loss: 32.22535705566406
epoch: 0, iter: 24600, loss: 33.123451232910156
epoch: 0, iter: 24700, loss: 43.97193908691406
epoch: 0, iter: 24800, loss: 31.96601676940918
epoch: 0, iter: 24900, loss: 32.94402313232422
epoch: 0, iter: 25000, loss: 32.07106399536133
epoch: 0, iter: 25100, loss: 31.70063018798828
epoch: 0, iter: 25200, loss: 43.262081146240234
epoch: 0, iter: 25300, loss: 32.8114013671875
epoch: 0, iter: 25400, loss: 44.05104064941406
epoch: 0, iter: 25500, loss: 32.40951156616211
epoch: 0, iter: 25600, loss: 32.1453857421875
epoch: 0, iter: 25700, loss: 31.737300872802734
epoch: 0, iter: 25800, loss: 32.191017150878906
epoch: 0, iter: 25900, loss: 32.642555236816406
epoch: 0, iter: 26000, loss: 32.354469299316406
epoch: 0, iteration: 26000, simlex-999: SpearmanrResu

epoch: 0, iter: 36100, loss: 32.15605163574219
epoch: 0, iter: 36200, loss: 31.857614517211914
epoch: 0, iter: 36300, loss: 31.209308624267578
epoch: 0, iter: 36400, loss: 31.760711669921875
epoch: 0, iter: 36500, loss: 32.03057098388672
epoch: 0, iter: 36600, loss: 32.61025619506836
epoch: 0, iter: 36700, loss: 31.615623474121094
epoch: 0, iter: 36800, loss: 32.07215118408203
epoch: 0, iter: 36900, loss: 31.55868911743164
epoch: 0, iter: 37000, loss: 31.60784912109375
epoch: 0, iter: 37100, loss: 31.788572311401367
epoch: 0, iter: 37200, loss: 31.19251823425293
epoch: 0, iter: 37300, loss: 31.588708877563477
epoch: 0, iter: 37400, loss: 31.62773895263672
epoch: 0, iter: 37500, loss: 31.803997039794922
epoch: 0, iter: 37600, loss: 31.85590171813965
epoch: 0, iter: 37700, loss: 31.738929748535156
epoch: 0, iter: 37800, loss: 32.09968948364258
epoch: 0, iter: 37900, loss: 43.67149353027344
epoch: 0, iter: 38000, loss: 31.698333740234375
epoch: 0, iteration: 38000, simlex-999: SpearmanrRe

epoch: 0, iter: 48100, loss: 32.016380310058594
epoch: 0, iter: 48200, loss: 31.03319549560547
epoch: 0, iter: 48300, loss: 31.599328994750977
epoch: 0, iter: 48400, loss: 32.17174530029297
epoch: 0, iter: 48500, loss: 32.302364349365234
epoch: 0, iter: 48600, loss: 31.845863342285156
epoch: 0, iter: 48700, loss: 31.601028442382812
epoch: 0, iter: 48800, loss: 32.54429626464844
epoch: 0, iter: 48900, loss: 31.173816680908203
epoch: 0, iter: 49000, loss: 32.889129638671875
epoch: 0, iter: 49100, loss: 32.68468475341797
epoch: 0, iter: 49200, loss: 32.787269592285156
epoch: 0, iter: 49300, loss: 30.99506378173828
epoch: 0, iter: 49400, loss: 31.552818298339844
epoch: 0, iter: 49500, loss: 31.03822898864746
epoch: 0, iter: 49600, loss: 29.724838256835938
epoch: 0, iter: 49700, loss: 31.63766098022461
epoch: 0, iter: 49800, loss: 31.447370529174805
epoch: 0, iter: 49900, loss: 32.101959228515625
epoch: 0, iter: 50000, loss: 30.738265991210938
epoch: 0, iteration: 50000, simlex-999: Spearma

...

epoch: 1, iter: 420100, loss: 31.26517105102539
epoch: 1, iter: 420200, loss: 29.995216369628906
epoch: 1, iter: 420300, loss: 31.235034942626953
epoch: 1, iter: 420400, loss: 31.15966033935547
epoch: 1, iter: 420500, loss: 30.632919311523438
epoch: 1, iter: 420600, loss: 30.661869049072266
epoch: 1, iter: 420700, loss: 30.54718017578125
epoch: 1, iter: 420800, loss: 30.1232852935791
epoch: 1, iter: 420900, loss: 30.441909790039062
epoch: 1, iter: 421000, loss: 30.62873077392578
epoch: 1, iter: 421100, loss: 30.611051559448242
epoch: 1, iter: 421200, loss: 31.97906494140625
epoch: 1, iter: 421300, loss: 30.443531036376953
epoch: 1, iter: 421400, loss: 30.15604019165039
epoch: 1, iter: 421500, loss: 30.163715362548828
epoch: 1, iter: 421600, loss: 31.048715591430664
epoch: 1, iter: 421700, loss: 30.81488037109375
epoch: 1, iter: 421800, loss: 30.606592178344727
epoch: 1, iter: 421900, loss: 30.099079132080078
epoch: 1, iter: 422000, loss: 30.300344467163086
epoch: 1, iteration: 422000, 

epoch: 1, iter: 432100, loss: 29.777925491333008
epoch: 1, iter: 432200, loss: 30.111785888671875
epoch: 1, iter: 432300, loss: 31.92470932006836
epoch: 1, iter: 432400, loss: 32.11894989013672
epoch: 1, iter: 432500, loss: 30.77094078063965
epoch: 1, iter: 432600, loss: 30.509416580200195
epoch: 1, iter: 432700, loss: 31.844635009765625
epoch: 1, iter: 432800, loss: 30.258201599121094
epoch: 1, iter: 432900, loss: 30.356861114501953
epoch: 1, iter: 433000, loss: 31.04130744934082
epoch: 1, iter: 433100, loss: 29.739765167236328
epoch: 1, iter: 433200, loss: 31.568098068237305
epoch: 1, iter: 433300, loss: 29.665851593017578
epoch: 1, iter: 433400, loss: 30.663497924804688
epoch: 1, iter: 433500, loss: 29.39330291748047
epoch: 1, iter: 433600, loss: 31.240802764892578
epoch: 1, iter: 433700, loss: 30.175386428833008
epoch: 1, iter: 433800, loss: 30.675495147705078
epoch: 1, iter: 433900, loss: 30.136138916015625
epoch: 1, iter: 434000, loss: 30.941184997558594
epoch: 1, iteration: 4340

epoch: 1, iter: 444100, loss: 29.231624603271484
epoch: 1, iter: 444200, loss: 30.372573852539062
epoch: 1, iter: 444300, loss: 31.155120849609375
epoch: 1, iter: 444400, loss: 30.369741439819336
epoch: 1, iter: 444500, loss: 31.307262420654297
epoch: 1, iter: 444600, loss: 31.125864028930664
epoch: 1, iter: 444700, loss: 30.759227752685547
epoch: 1, iter: 444800, loss: 30.860876083374023
epoch: 1, iter: 444900, loss: 29.877044677734375
epoch: 1, iter: 445000, loss: 30.604446411132812
epoch: 1, iter: 445100, loss: 30.765357971191406
epoch: 1, iter: 445200, loss: 29.81828498840332
epoch: 1, iter: 445300, loss: 30.53061294555664
epoch: 1, iter: 445400, loss: 30.37859535217285
epoch: 1, iter: 445500, loss: 28.88435173034668
epoch: 1, iter: 445600, loss: 29.322994232177734
epoch: 1, iter: 445700, loss: 30.45917320251465
epoch: 1, iter: 445800, loss: 31.456871032714844
epoch: 1, iter: 445900, loss: 30.015295028686523
epoch: 1, iter: 446000, loss: 30.787403106689453
epoch: 1, iteration: 4460

epoch: 1, iter: 456100, loss: 30.92245864868164
epoch: 1, iter: 456200, loss: 29.203149795532227
epoch: 1, iter: 456300, loss: 29.316104888916016
epoch: 1, iter: 456400, loss: 30.696075439453125
epoch: 1, iter: 456500, loss: 31.72907257080078
epoch: 1, iter: 456600, loss: 30.80364227294922
epoch: 1, iter: 456700, loss: 31.051776885986328
epoch: 1, iter: 456800, loss: 29.763452529907227
epoch: 1, iter: 456900, loss: 31.195384979248047
epoch: 1, iter: 457000, loss: 29.543630599975586
epoch: 1, iter: 457100, loss: 28.87030792236328
epoch: 1, iter: 457200, loss: 30.461898803710938
epoch: 1, iter: 457300, loss: 29.940473556518555
epoch: 1, iter: 457400, loss: 30.70020866394043
epoch: 1, iter: 457500, loss: 29.576948165893555
epoch: 1, iter: 457600, loss: 30.57088851928711
epoch: 1, iter: 457700, loss: 31.206768035888672
epoch: 1, iter: 457800, loss: 29.944683074951172
epoch: 1, iter: 457900, loss: 31.825054168701172
epoch: 1, iter: 458000, loss: 30.528287887573242
epoch: 1, iteration: 45800

epoch: 1, iter: 468100, loss: 29.94235610961914
epoch: 1, iter: 468200, loss: 31.483890533447266
epoch: 1, iter: 468300, loss: 31.022171020507812
epoch: 1, iter: 468400, loss: 31.710355758666992
epoch: 1, iter: 468500, loss: 30.432117462158203
epoch: 1, iter: 468600, loss: 30.19153594970703
epoch: 1, iter: 468700, loss: 30.63540267944336
epoch: 1, iter: 468800, loss: 30.547693252563477
epoch: 1, iter: 468900, loss: 30.727825164794922
epoch: 1, iter: 469000, loss: 30.90317153930664
epoch: 1, iter: 469100, loss: 31.289710998535156
epoch: 1, iter: 469200, loss: 30.831520080566406
epoch: 1, iter: 469300, loss: 31.321063995361328
epoch: 1, iter: 469400, loss: 30.700862884521484
epoch: 1, iter: 469500, loss: 31.952341079711914
epoch: 1, iter: 469600, loss: 30.4019775390625
epoch: 1, iter: 469700, loss: 29.877962112426758
epoch: 1, iter: 469800, loss: 29.757856369018555
epoch: 1, iter: 469900, loss: 29.875370025634766
epoch: 1, iter: 470000, loss: 31.016456604003906
epoch: 1, iteration: 47000

In [11]:
###在 MEN 和 Simplex-999 数据集上做评估
model.load_state_dict(torch.load("embedding-{}.th".format(EMBEDDING_SIZE)))
embedding_weights = model.input_embeddings()
print("simlex-999", evaluate("E:\\simlex-999.txt", embedding_weights))
print("men", evaluate("E:\\men.txt", embedding_weights))
print("wordsim353", evaluate("E:\\wordsim353.csv", embedding_weights))

simlex-999 SpearmanrResult(correlation=0.19078792185881202, pvalue=2.695910502724768e-09)
men SpearmanrResult(correlation=0.40737158067232526, pvalue=3.6861885533198073e-104)
wordsim353 SpearmanrResult(correlation=0.3958016698968927, pvalue=2.2684035915462937e-13)


In [12]:

for word in ["good", "fresh", "monster", "green", "like", "america", "chicago", "work", "computer", "language"]:
    print(word, find_nearest(word))

good ['good', 'really', 'worthless', 'everything', 'bad', 'how', 'foolish', 'thus', 'even', 'tangible']
fresh ['fresh', 'drinking', 'waste', 'food', 'nutrients', 'fruit', 'grain', 'clean', 'ingesting', 'soaked']
monster ['monster', 'loch', 'dove', 'firefly', 'mysterious', 'creature', 'sting', 'buff', 'feather', 'eye']
green ['green', 'purple', 'white', 'red', 'blue', 'grass', 'yellow', 'wings', 'striped', 'brown']
like ['like', 'hams', 'resembling', 'or', 'comical', 'whose', 'unlike', 'producing', 'resemble', 'very']
america ['america', 'africa', 'europe', 'india', 'south', 'england', 'australia', 'borneo', 'west', 'mexico']
chicago ['chicago', 'boston', 'richmond', 'illinois', 'campus', 'florida', 'cincinnati', 'detroit', 'seattle', 'atlanta']
work ['work', 'unsurprisingly', 'talent', 'writing', 'upbringing', 'seminal', 'works', 'composing', 'audience', 'job']
computer ['computer', 'hardware', 'gaming', 'manuals', 'computing', 'designing', 'advanced', 'simulation', 'software', 'consol

In [13]:
man_idx = word_to_idx["man"] 
king_idx = word_to_idx["king"] 
woman_idx = word_to_idx["woman"]
embedding = embedding_weights[woman_idx] - embedding_weights[man_idx] + embedding_weights[king_idx]
cos_dis = np.array([scipy.spatial.distance.cosine(e, embedding) for e in embedding_weights])
for i in cos_dis.argsort()[:20]:
    print(idx_to_word[i])

king
woman
henry
edward
joshua
margaret
catherine
becket
marries
bartholomew
knight
george
theresa
xiv
james
stuart
john
carloman
gregory
daughter
